In [1]:
## https://dev.mysql.com/downloads/file/?id=541637   MySQL Community SetupFile Download LINK

In [32]:
## Importing Libraries
import mysql.connector
import pandas as pd

In [33]:
## Setting Connection to Mysql Server
conn = mysql.connector.connect(
    host='localhost',         
    user='root',   
    password='0980'
)
print(conn)

In [34]:
## Display DATASET
mycursor = conn.cursor()
mycursor.execute("SHOW DATABASES")
for x in mycursor:
  print(x)

('information_schema',)
('mysql',)
('performance_schema',)
('sakila',)
('sys',)
('world',)


In [35]:
## Creating DataBase
mycursor.execute("CREATE DATABASE Discom_Drawal")

In [36]:
## Checking if the DataBase Created or Not
mycursor.execute("SHOW DATABASES")
for x in mycursor:
  print(x)

('discom_drawal',)
('information_schema',)
('mysql',)
('performance_schema',)
('sakila',)
('sys',)
('world',)


In [37]:
mycursor.execute("USE Discom_Drawal")

In [38]:
mycursor.execute("SHOW TABLES")
for x in mycursor:
  print(x)

In [39]:
## Loading Dataset
df = pd.read_csv('Final.csv')

In [40]:
df.head()

,TimeDesc,Bazpur Solar Plant,Bhilangana Hydro Power Ltd.,Chibro Power House,Chilla Power House,Dhakrani Power House,Dhalipur Power House,Dharasu Power House,Gaderpur Solar Plant,Gama Infroprop (p) Ltd.,...,Eminent solar power pvt ltd,Sunworld energy pvt ltd,Ballupur solar power project pvt ltd,Chudiala solar power project pvt ltd,Shakumbhari solar power project pvt ltd,Haridwar solar project pvt ltd 1,Haridwar solar project pvt ltd 2,IaSGS,ISGS,Total
0,2020-05-17 00:00:00,0.0,24.0,120.0,95.0,24.0,17.0,76.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,593.2,586.93,1180.13
1,2020-05-17 00:15:00,0.0,24.0,120.0,95.0,23.0,17.0,76.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,591.2,556.81,1148.01
2,2020-05-17 00:30:00,0.0,24.0,120.0,95.0,23.0,17.0,76.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,591.2,541.66,1132.86
3,2020-05-17 00:45:00,0.0,24.0,120.0,95.0,23.0,17.0,76.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,595.2,511.22,1106.42
4,2020-05-17 01:00:00,0.0,24.0,120.0,95.0,23.0,17.0,76.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,595.2,508.39,1103.59


In [48]:
len(df)

175296

In [41]:
column_types = {
    'int64': 'INT',
    'float64': 'FLOAT',
    'object': 'VARCHAR(255)',
    'datetime64[ns]': 'DATETIME'
}

In [42]:
## Create table with TimeDesc as primary key
columns = []
for col_name, dtype in df.dtypes.items():
    sql_type = column_types.get(str(dtype), 'VARCHAR(255)')
    if col_name == 'TimeDesc':
        columns.append(f"`{col_name}` {sql_type} PRIMARY KEY")
    else:
        columns.append(f"`{col_name}` {sql_type}")

create_table_sql = f"""
CREATE TABLE DISCOM_DRAWAL_DATA (
    {', '.join(columns)}
)
"""

In [43]:
mycursor.execute(create_table_sql)

In [44]:
## Prepare data for insertion
insert_query = f"""
INSERT INTO DISCOM_DRAWAL_DATA ({', '.join(f'`{col}`' for col in df.columns)})
VALUES ({', '.join(['%s'] * len(df.columns))})
"""
data = [tuple(row) for row in df.itertuples(index=False)]

In [23]:
## Clearing Result if any Displayed or Pending
mycursor.fetchall() 

In [45]:
##INSERTING DATA
try:
    mycursor.executemany(insert_query, data)
    conn.commit()
    print("Data inserted successfully.")
except mysql.connector.Error as err:
    print("Error inserting data:", err)

Data inserted successfully.


In [47]:
mycursor.execute("SELECT COUNT(*) FROM DISCOM_DRAWAL_DATA;")
for x in mycursor:
    print(x)

(175296,)


In [49]:
# Finalize
conn.commit()
mycursor.close()
conn.close()